### POP with cubic reqularization

In [1]:
using DynamicPolynomials
using LinearAlgebra
include("LiPoSID.jl")
using Dates
using HDF5

$
    \frac{d\rho}{dt} = -\frac{i}{\hbar} [H,\rho(t)] + \mathcal{D}[\rho(t)] = - \frac{i}{\hbar}[H, \rho]+\sum_{\ell=1}^{s-1}\left[A_\ell \rho A_\ell^\dagger - \frac{1}{2}\left\{ A_\ell^\dagger A_\ell, \rho \right\} \right]
$

In [2]:
#using DynamicPolynomials

@polyvar e[1:2]
@polyvar h[1:2]


Hˢʸᵐᵇ = [ e[1]               h[1] - im*h[2]
          h[1] + im*h[2]     0.             ]

2×2 Matrix{Polynomial{true, ComplexF64}}:
 e₁                  h₁ + (0.0-1.0im)h₂
 h₁ + (0.0+1.0im)h₂  0.0+0.0im

In [3]:
@polyvar a[1:3]
@polyvar b[1:3]


Aˢʸᵐᵇ = [ a[1] + im*b[1]           a[2] + im*b[2]
          a[3] + im*b[3]          -a[1] - im*b[1]   ]

2×2 Matrix{Polynomial{true, Complex{Int64}}}:
 a₁ + (0+1im)b₁  a₂ + (0+1im)b₂
 a₃ + (0+1im)b₃  -a₁ + (0-1im)b₁

In [4]:
parentdir = pwd()
data_dir = parentdir*"\\DATA\\"

"C:\\Users\\Zakhar\\Documents\\GitHub\\POP_fail\\DATA\\"

In [5]:
output_directory = parentdir*"\\MODELS\\"

"C:\\Users\\Zakhar\\Documents\\GitHub\\POP_fail\\MODELS\\"

$
 L^{(Lindblad)}
= \sum_{i=1}^N {
{\left\|
\rho_{(i)} - \rho_{(i-2)}
- {\mathcal{L}} \left[\int^{t_i}_{t_{i-2}}\rho(t)dt \right] \right\| }^2_F }
$

$
\int^{t_i}_{t_{i-2}}\rho(t)dt = \frac{1}{3} \Delta t \left[ {\rho(t_{i-2}) + 4 \rho(t_{i-1}) + \rho(t_i)} \right] +  \mathcal{O}(\Delta t^5). \label{eq:simpson}
$

In [6]:
date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))
res_file_name = "POP_2CUT_train-B_reg3_0001" * date_and_time_string * ".h5"

Eˢⁱᵈ = []
γˢⁱᵈ = []

γᵗˣᵗ = [ "0.079477"] #, "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

@time for γᵢᵗˣᵗ in γᵗˣᵗ
    
    println("γ = ", γᵢᵗˣᵗ)
    
    ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", γᵢᵗˣᵗ)
    ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B2_2CUT_data.h5", γᵢᵗˣᵗ)
    ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_2CUT_data.h5", γᵢᵗˣᵗ)
    ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_2CUT_data.h5", γᵢᵗˣᵗ)
    
    tˢᵗᵉᵖₘₐₓ = min(length(ρᵍ), length(ρᵉ), length(ρˣ), length(ρʸ)) 

    @assert tᵍ[1:tˢᵗᵉᵖₘₐₓ] == tᵉ[1:tˢᵗᵉᵖₘₐₓ] == tˣ[1:tˢᵗᵉᵖₘₐₓ] == tʸ[1:tˢᵗᵉᵖₘₐₓ] 
    t = convert(Vector{Float64}, tᵉ)
    @assert maximum(diff(t)) ≈ minimum(diff(t)) ≈ t[2]-t[1]
    
    obj = 0

    for ρ in [ρᵍ[1:tˢᵗᵉᵖₘₐₓ], ρᵉ[1:tˢᵗᵉᵖₘₐₓ], ρˣ[1:tˢᵗᵉᵖₘₐₓ], ρʸ[1:tˢᵗᵉᵖₘₐₓ]]

        ρ = convert(Vector{Matrix{ComplexF64}}, ρ)
        obj += LiPoSID.simpson_obj(ρ, t,  Hˢʸᵐᵇ, [Aˢʸᵐᵇ])

    end # of files (initial states) loop 
    
    solution_simp, best_method_simp = LiPoSID.sos_min_newton(obj + 0.001 * sum(a.^3 + b.^3) )
    
    Hˢⁱᵈ = subs(Hˢʸᵐᵇ, solution_simp)
    Aˢⁱᵈ = subs(Aˢʸᵐᵇ, solution_simp)
    
    h5open(output_directory*res_file_name,"cw") do fid  # read-write, create file if not existing, preserve existing contents

       γ_group = create_group(fid, "gamma_"*string(γᵢᵗˣᵗ))

        γ_group["H"] = convert.(ComplexF64, Hˢⁱᵈ )
        γ_group["A"] = convert.(ComplexF64, Aˢⁱᵈ )

        end # of HDF5 writing
    
    println()

end # of loop over γ

γ = 0.079477
************************TSSOS************************
TSSOS is launching...
optimum = 0.0005252831278639165

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Global optimality certified!
No higher TSSOS hierarchy!
************************TSSOS************************
TSSOS is launching...
optimum = 0.0005252104680473283
Global optimality certified!
No higher TSSOS hierarchy!
val_p = [0.0005252240571280709, 0.0005252239063224806]

γ = 0.25133
************************TSSOS************************
TSSOS is launching...
optimum = 0.0008165415602671289
Global optimality certified!
No higher TSSOS hierarchy!
************************TSSOS***********